In [1]:
import requests
import os
import json
from api_keys import BEARER_TOKEN
from tqdm import tqdm
import os
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re


[nltk_data] Downloading package stopwords to /home/julio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from unicodedata import normalize as norm

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str).encode('ASCII', 'ignore')
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


In [ ]:
unicodedata.normalize('NFKD', "mãe").encode('ASCII', 'ignore')

In [ ]:
norm('NFKD', "mãe").encode('ascii', 'ignore').decode().lower()

In [ ]:
text = "https:4//t.co/lmvoS5uwFd poxa globoplay...  https:3//t.co/lmvoS5uwFd   https:2//t.co/lmvoS5uwFd https:1//t.co/lmvoS5uwFd   "

In [ ]:

regex_remove_https = 'https([a-zA-Zà-úÀ-Ú0-9]|[-()\#/@;:<>{}`+=~|.!?,])+'
#text_without_https = re.sub(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https)," ",text)
#print("\n")
#print("\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https))
#print("\n")

text_without_https = re.sub(r"(\s|^){0}(\s{0})*($|\s)".format(regex_remove_https)," ",text)

In [ ]:
text_without_https

In [ ]:
if re.search(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https),text_without_https):
    
    print("1")

In [ ]:
re.findall(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https),text,re.MULTILINE)

# Funções (Análise Exploratória)

### Função para plotar bar plot com a contagem de tokens

In [ ]:
def plot_bar_count_words(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = CountVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para plotar bar plot com tf-idf

In [ ]:
def plot_bar_tf_idf(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = TfidfVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    "MAX":data_vect.std(axis=0)
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para contagem de tokens

In [ ]:
def calculate_number_words(text):

    quantity_of_words = text.split(" ")

    quantity_of_words = [i for i in quantity_of_words if i!=""]

    quantity_of_words = len(quantity_of_words)

    return quantity_of_words


### Função para contagem de diferentes tokens

In [ ]:
def calculate_number_diferent_words(text):

    quantity_of_diferent_words = text.split(" ")

    quantity_of_diferent_words = [i for i in quantity_of_diferent_words if i!=""]

    quantity_of_diferent_words = set(quantity_of_diferent_words)

    quantity_of_diferent_words = list(quantity_of_diferent_words)

    quantity_of_diferent_words = len(quantity_of_diferent_words)

    return quantity_of_diferent_words


### Função para criar textos sem repetição de palavras para ser utilizado na análise exploratória 

In [ ]:
def convert_text_to_no_repeat_words(text):

    text_with_no_repeat_words = text.split(" ")

    text_with_no_repeat_words = [i for i in text_with_no_repeat_words if i!=""]

    text_with_no_repeat_words = set(text_with_no_repeat_words)

    text_with_no_repeat_words = list(text_with_no_repeat_words)

    text_with_no_repeat_words = " ".join(text_with_no_repeat_words)

    return text_with_no_repeat_words

### Função para o pré-processamento do texto 

In [ ]:
def text_cleaner(text):
    
    nltk_stopwords = stopwords.words('portuguese')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

### Função para limpeza dos textos

In [ ]:
def text_cleaner(text,stop_words_domain =None):
    
    nltk_stopwords = stopwords.words('portuguese')
    regex_stop_words = '|'.join(nltk_stopwords)
    text = re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s"," ",text)
    text = re.sub(r"[^a-zA-ZÀ-Úà-ú]+"," ",text)
    text = re.sub(r"\s({})\s|\s({})$|^({})\s".format(regex_stop_words,regex_stop_words,regex_stop_words)," ",text)
    
    return text

In [ ]:
txt = " uhahua,não,"

In [ ]:
text_cleaner(txt)

In [ ]:
print(stopwords.words('portuguese'))

# Funções (Extração de Tweets)

In [23]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(query = "@globoplay -is:retweet",until_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&until_id={}&tweet.fields=author_id,created_at".format(
            query,until_id
        )
        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&tweet.fields=author_id,created_at".format(
            query
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = "@BBB -is:retweet",until_id=None):
    bearer_token = BEARER_TOKEN
    url = create_url(query,until_id)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    
    print("\n")
    print(json_response)
    print("\n")
    
    return json_response

def extract_many_tweets(qnt_cycle=10,folder="data_tweets",until_id=None,query="@BBB"):
    
    
    oldest_id = None
    
    newest_id = None
    
    
    
    for i in tqdm(range(qnt_cycle)):
    
        
        if i == 0:
            
            #extract the 100 tweets first
            
            if not start_from_id:
        
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=None)
            
            else:
                
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=until_id)
                
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            #get the current date
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            newest_id = data_tweets['meta']['newest_id']
            
            oldest_date = date_extraction
            
            df_data_tweets = df_data_tweets_temp.copy()
            
            # name file
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file,sep=",")
            
    
            
        else:
            
            
            #extract more 100 tweets older
            
            data_tweets_temp = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=oldest_id)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets_temp["data"])
            
            
            #get the current date
            
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            oldest_id = data_tweets_temp['meta']['oldest_id']
            
            df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])
            
            date_extraction = datetime.now()
            
            df_data_tweets.reset_index(inplace=True,drop=True)
            
            
            # remove old files
            
            os.remove(name_file)
            
            
            # name file
            
            oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file.format(folder),sep=",")
            
            

    
    return df_data_tweets
    
    


# Extração de Tweets

In [24]:
data_tweets_final = extract_many_tweets(qnt_cycle=1,folder="data_tweets",query="globoplay")#,start_from_id="1367600965277384706")

100%|██████████| 1/1 [00:01<00:00,  1.06s/it]



{'data': [{'created_at': '2021-04-24T02:25:19.000Z', 'text': 'Reprodução: Globo/@globoplay', 'id': '1385781871930445831', 'author_id': '1274033632286572544'}, {'created_at': '2021-04-24T02:24:49.000Z', 'text': 'nem digo quem foi... @globoplay https://t.co/fAx5TgZzXr', 'id': '1385781745656766465', 'author_id': '2158642561'}, {'created_at': '2021-04-24T02:24:40.000Z', 'text': 'Toda vez que ela aparece no comercial meu coração erra uma batida 🎥 @globoplay  https://t.co/vRNnF6hSMX', 'id': '1385781708834869250', 'author_id': '4708156587', 'attachments': {'media_keys': ['7_1381261308776095751']}}, {'created_at': '2021-04-24T02:24:37.000Z', 'text': 'Cara na real? Nesta última semana acompanhando o @PauloVieiraReal nos "breaks" da @globoplay é tão bacana ver ele ganhando espaço, ele trás a informação com leveza tão boa e não é por ser humorista é por ser talentoso real.', 'id': '1385781696654610441', 'author_id': '1862871385'}, {'created_at': '2021-04-24T02:24:30.000Z', 'text': 'Globoplay: i

In [25]:
data_tweets_final

,created_at,text,id,author_id,attachments,date_extraction
0,2021-04-24T02:25:19.000Z,Reprodução: Globo/@globoplay,1385781871930445831,1274033632286572544,NaN,2021-04-23 23:25:41.907118
1,2021-04-24T02:24:49.000Z,nem digo quem foi... @globoplay https://t.co/f...,1385781745656766465,2158642561,NaN,2021-04-23 23:25:41.907118
2,2021-04-24T02:24:40.000Z,Toda vez que ela aparece no comercial meu cora...,1385781708834869250,4708156587,{'media_keys': ['7_1381261308776095751']},2021-04-23 23:25:41.907118
3,2021-04-24T02:24:37.000Z,Cara na real? Nesta última semana acompanhando...,1385781696654610441,1862871385,NaN,2021-04-23 23:25:41.907118
4,2021-04-24T02:24:30.000Z,Globoplay: investindo no break do BBB.\nCriati...,1385781666484981760,3572644696,NaN,2021-04-23 23:25:41.907118
5,2021-04-24T02:24:26.000Z,@vitorvazdf amiga eh pq no globoplay da pra ve...,1385781649510739973,68737051,{'media_keys': ['7_1385766509528633344']},2021-04-23 23:25:41.907118
6,2021-04-24T02:24:25.000Z,📽️REPRODUÇÃO TV GLOBO // GLOBOPLAY,1385781643961643010,1323848059642667009,NaN,2021-04-23 23:25:41.907118
7,2021-04-24T02:24:23.000Z,"Só a namaria mesmo pra juntar youtube, netflix...",1385781635652722692,435139870,NaN,2021-04-23 23:25:41.907118
8,2021-04-24T02:24:17.000Z,@inevithabie assinaria o globoplay por essa pr...,1385781612277911555,887481441365504000,NaN,2021-04-23 23:25:41.907118
9,2021-04-24T02:24:13.000Z,"Ah, Globoplay, quem não te conhece que te comp...",1385781593986453512,1383567473346187264,NaN,2021-04-23 23:25:41.907118


In [14]:
data_tweets_final["text"][0]

'@annytavz @kayrsp @citeserieefilme @globoplay @YouTubeBrasil @NetflixBrasil @rafaelportugal que ódio kkkkkk'

In [15]:
data_tweets_final["text"][101]

'@globoplay Eu ainda não acredito que vocês fizeram isso... \n\nPelo amor de Deus. \n\nEu não vou perder meu tempo. \nE pra mim tomara que perda o investimento. \n\nSó fez maldade... maldosa mesmo.. a fama dela com relacionamentos com pessoas já era ruim aqui fora.. ai mudou do nada. NAO ACREDITO'

In [ ]:
data_tweets_final["text_clean"] = data_tweets_final["text"].apply(lambda x: text_cleaner(text = x,stop_words_domain =None))

In [ ]:
data_tweets_final[["text","text_clean"]]

In [ ]:
re.sub(,data_tweets_final["text"][3]

In [ ]:
data_tweets_final["text"][3]

In [ ]:
test = 'https://t.co/HaKu8sRLkX Melhor pessoa desse bbb '

In [ ]:
re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s"," ",test)

In [ ]:
re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s"," ",test)

In [ ]:
https([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s

In [ ]:
def function_to_calc_histogram(x,initial_interval, final_interval,n_bins,indice=False):

    interval = np.linspace(initial_interval, final_interval, num=n_bins)

    for j,i in enumerate(interval):


        if i == interval[len(interval)-1]:

            if x>=i:

                
                if indice:

                    return j
                
                else:
                    
                    return "{}<".format(x)


        else:

            if x>=i and x<interval[j+1]:

                inicial = round(i, 1)

                final = round(interval[j+1],1)

            
                if indice:

                    return j
                
                else:
                    
                    return "[{},{})".format(inicial,final)


# Criação de uma coluna com os textos sem repetição de palavras para ser utilizado na análise exploratória

In [ ]:
data_tweets_final['text_unique_words'] = data_tweets_final['text'].apply(lambda x: convert_text_to_no_repeat_words(x))

# Calculo Número de tokens

In [ ]:
data_tweets_final['number_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_words(x))

# Calculo Número de diferentes tokens

In [ ]:
data_tweets_final['number_diferent_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_diferent_words(x))

# Máximo número de tokens

In [ ]:
max_count = data_tweets_final["number_tokens"].max()

In [ ]:
data_tweets_final["number_tokens"].var()

# Mínimo número de tokens

In [ ]:
min_count = data_tweets_final["number_tokens"].min()

## Dados para o histograma 

In [ ]:
data_tweets_final['bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=False))

In [ ]:
data_tweets_final['indices_bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=True))

In [ ]:
data_histogram = data_tweets_final.groupby(["bins","indices_bins"]).sum().sort_values(by=["indices_bins"])

In [ ]:
data_histogram.reset_index(drop=False,inplace=True)

In [ ]:
data_histogram

In [ ]:
data_histogram["bins"].tolist()

In [ ]:
data_histogram["number_tokens"].tolist()

# DF top 10 MEAN

In [ ]:
df_report_mean = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [ ]:
df_report_mean["MEAN"].tolist()

In [ ]:
df_report_mean["WORDS"].tolist()

# DF top 10 SUM docs

In [ ]:
df_report_sum = plot_bar_count_words(text_column='text_unique_words',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [ ]:
df_report_sum["P_DOCS"] =  df_report_sum["SUM"]/len(data_tweets_final)

In [ ]:
df_report_sum

# DF top 10 SUM

In [ ]:
df_report_sum_docs = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [ ]:
df_report_sum_docs

# DF top 10 MEAN TF-IDF

In [ ]:
df_report_tfidf_mean = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [ ]:
df_report_tfidf_mean

# DF top 10 MAX TF-IDF

In [ ]:
df_report_tfidf_max = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MAX',top=10,return_df=True)

In [ ]:
df_report_tfidf_max